# Allocation Algorithm 1

> **1.** For each EI in list, assign the child that is closest to that EI along with any other children also at that address.

> **2.** Remove all of the children at that location from the list of available children.

> **3.** After each iteration through the EIs, pull the number of children each EI has assigned to them.  All of the EIs that have the lowest number of children in the list are fed into the next iteration of EIs for assignment to ensure all the EIs have a mostly even number of children for a balanced dataset.

> **4.** Continue this process until all children have been assigned to an EI

> **5.** Update MySQL table locationdataalgo1

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

In [3]:
query = ("UPDATE child "
         "SET assigned = 0")
cursor.execute(query)

In [4]:
query = ("SELECT ei_id FROM ei "
         "WHERE ei_id != '65d75e28e09e67e025e8cd3c'")
cursor.execute(query)

ei_ids = cursor.fetchall()

for row in ei_ids:
    print(row[0])

65d75a2ce09e67e025e8cd30
65d75bb0e09e67e025e8cd31
65d75c59e09e67e025e8cd32
65d75c87e09e67e025e8cd33
65d75cb9e09e67e025e8cd34
65d75ce1e09e67e025e8cd35
65d75d08e09e67e025e8cd36
65d75d4ce09e67e025e8cd37
65d75d80e09e67e025e8cd38
65d75da9e09e67e025e8cd39
65d75dd5e09e67e025e8cd3a
65d75dfae09e67e025e8cd3b


In [5]:
column_names = ['ei_id']

# Create DataFrame
eiDF = pd.DataFrame(ei_ids, columns=column_names)
eiDF["assignment_count"] = ""
print(eiDF)

                       ei_id assignment_count
0   65d75a2ce09e67e025e8cd30                 
1   65d75bb0e09e67e025e8cd31                 
2   65d75c59e09e67e025e8cd32                 
3   65d75c87e09e67e025e8cd33                 
4   65d75cb9e09e67e025e8cd34                 
5   65d75ce1e09e67e025e8cd35                 
6   65d75d08e09e67e025e8cd36                 
7   65d75d4ce09e67e025e8cd37                 
8   65d75d80e09e67e025e8cd38                 
9   65d75da9e09e67e025e8cd39                 
10  65d75dd5e09e67e025e8cd3a                 
11  65d75dfae09e67e025e8cd3b                 


In [6]:
query = ("SELECT child_id from child")
cursor.execute(query)

child_ids = cursor.fetchall()

for row in child_ids:
    print(row[0])

65d76430e09e67e025e8cd43
65d764dde09e67e025e8cd44
65d766f6e09e67e025e8cd45
65d76799e09e67e025e8cd46
65d76829e09e67e025e8cd47
65d76956e09e67e025e8cd48
65d769fde09e67e025e8cd49
65d76b1ee09e67e025e8cd4a
65d76bb7e09e67e025e8cd4b
65d76c25e09e67e025e8cd4c
65d76ca3e09e67e025e8cd4d
65d76d38e09e67e025e8cd4e
65d76dcae09e67e025e8cd4f
65d76e6de09e67e025e8cd50
65d76f01e09e67e025e8cd51
65d76f7ae09e67e025e8cd52
65d76ff5e09e67e025e8cd53
65d77069e09e67e025e8cd54
65d770fde09e67e025e8cd55
65d771a3e09e67e025e8cd5a
65d7725be09e67e025e8cd5b
65d772ffe09e67e025e8cd5c
65d773a1e09e67e025e8cd5d
65d77c78e09e67e025e8cd5e
65d77d33e09e67e025e8cd5f
65d77ed6e09e67e025e8cd60
65d77f4de09e67e025e8cd61
65d77fdce09e67e025e8cd62
65d78068e09e67e025e8cd63
65d780cee09e67e025e8cd64
65d78173e09e67e025e8cd65
65d78213e09e67e025e8cd66
65d782a0e09e67e025e8cd67
65d7830de09e67e025e8cd68
65d78390e09e67e025e8cd69
65d7842ee09e67e025e8cd6a
65d78493e09e67e025e8cd6b
65d7852de09e67e025e8cd6c
65d785dee09e67e025e8cd6d
65d7864fe09e67e025e8cd6e


In [7]:
def runAlgo1(ei_ids, eiDF):
    for ei in ei_ids:
        query = ("Select ld.child_id_1, ld.ei_id_1, ld.assigned, ld.driving_distance, ld.driving_time, child.Longitude, child.Latitude, child.assigned from locationdataalgo1 ld "
                 "join child on child.child_id=ld.child_id_1 "
                 f"where ld.child_id_2 is null and ld.ei_id_2 is null and child.assigned = 0 and ei_id_1 = '{ei}' "
                 "order by driving_distance asc")
        cursor.execute(query)
        ei_data = cursor.fetchall()

        for child in ei_data:
            print(child)
            print(child[5])
            print(child[6])
            longitude = child[5]
            latitude = child[6]
            break

        updateQuery = ("UPDATE locationdataalgo1 ld "
                       "JOIN child on ld.child_id_1 = child.child_id " 
                       f"SET ld.assigned = 1, child.assigned = 1 " 
                       f"WHERE child.longitude = {longitude} and child.latitude = {latitude} and ld.ei_id_1 = '{ei}'")
        cursor.execute(updateQuery)
        connection.commit()
        print("Update query pushed")

    countQuery = ("Select ei_id_1, count(*) from locationdataalgo1 "
                  "where assigned = 1 "
                  "group by ei_id_1")
    cursor.execute(countQuery)
    countByEI = cursor.fetchall()

    for ei_count in countByEI:
        eiDF.loc[eiDF['ei_id'] == ei_count[0], 'assignment_count'] = ei_count[1]
    print(eiDF)

    min_count = eiDF['assignment_count'].min()
    ei_ids_with_min_count = eiDF.loc[eiDF['assignment_count'] == min_count, 'ei_id'].tolist()
    print(ei_ids_with_min_count)
    
    return ei_ids_with_min_count, eiDF

In [8]:
minEIs, eiDF = runAlgo1(ei_ids, eiDF)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '65d75a2ce09e67e025e8cd30',)' order by driving_distance asc' at line 1

In [ ]:
print(type(ei_ids))
print(type(minEIs))
print(ei_ids[0])
print(minEIs[0])

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
minEIs, eiDF = runAlgo1(minEIs, eiDF)

In [ ]:
cursor.close()
connection.close()